In [1]:
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from datasets import DatasetDict

In [2]:
#from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import evaluate
import torch
import numpy as np
from transformers import AutoModelForCausalLM


In [2]:
model_checkpoint = "facebook/galactica-125M"
model_checkpoint = "facebook/galactica-1.3b"
lr = 1e-3
batch_size = 128
num_epochs = 3

In [3]:
datasets = load_dataset('flax-sentence-embeddings/stackexchange_math_jsonl')
#datasets = load_dataset('/home/sslashinin/.cache/huggingface/datasets/flax-sentence-embeddings___stackexchange_math_jsonl/title_answer/1.1.0/6d8702724ae9d14595d55b555648bb7a4909ce0441581f0c5a81e742633441a6')

No config specified, defaulting to: stackexchange_math_jsonl/title_answer
Found cached dataset stackexchange_math_jsonl (/home/sslashinin/.cache/huggingface/datasets/flax-sentence-embeddings___stackexchange_math_jsonl/title_answer/1.1.0/6d8702724ae9d14595d55b555648bb7a4909ce0441581f0c5a81e742633441a6)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'upvoted_answer'],
        num_rows: 1100953
    })
})

In [5]:
used_dataset, unused_dataset= datasets['train'].train_test_split(test_size=0.8).values()

In [6]:
used_dataset

Dataset({
    features: ['title', 'upvoted_answer'],
    num_rows: 220190
})

In [7]:
#train_dataset, testval_dataset= datasets['train'].train_test_split(test_size=0.2).values()
train_dataset, testval_dataset = used_dataset.train_test_split(test_size=0.5).values()

In [8]:
test_dataset, val_dataset = testval_dataset.train_test_split(test_size=0.5).values()

In [9]:
datasets_full = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': val_dataset})

In [10]:
datasets_full['train']

Dataset({
    features: ['title', 'upvoted_answer'],
    num_rows: 110095
})

In [11]:
datasets_full

DatasetDict({
    train: Dataset({
        features: ['title', 'upvoted_answer'],
        num_rows: 110095
    })
    test: Dataset({
        features: ['title', 'upvoted_answer'],
        num_rows: 55047
    })
    validation: Dataset({
        features: ['title', 'upvoted_answer'],
        num_rows: 55048
    })
})

In [12]:
datasets_full["train"][10]

{'title': 'Compute integral with residue theorem',
 'upvoted_answer': 'Let, \n\n$$I(a)=\\int_{-\\infty}^{\\infty}\\frac{\\sin^2(ax)dx}{x^2}$$  \n\n$$\\frac{dI}{da}=\\int_{-\\infty}^{\\infty}\\frac{\\sin(2ax)dx}{x}=\\pi$$  \n\n$$ I(a)=\\pi a+ C$$  \n\n$$I(a)=\\pi a$$ because $I(0)=0$.Hence for your particular case the answer is $ \\pi /2$'}

In [13]:
#seqeval = evaluate.load("seqeval")

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [15]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["upvoted_answer"]], truncation=True)

In [16]:
tokenized_datasets = datasets_full.map(preprocess_function, batched=True, num_proc=4, remove_columns=datasets_full["train"].column_names)

Map (num_proc=4):   0%|          | 0/110095 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map (num_proc=4):   0%|          | 0/55047 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map (num_proc=4):   0%|          | 0/55048 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    #for k in examples.keys():
    #  print(k)
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [18]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/110095 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Process ForkPoolWorker-17:


TimeoutError: 

In [20]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [67]:
# peft_config = LoraConfig(
#     task_type="CAUSAL_LM", inference_mode=False, r=16, lora_alpha=16, lora_dropout=0.1, bias="all"
# )

In [68]:
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

trainable params: 3688448 || all params: 1318346752 || trainable%: 0.27977829007462823


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [46]:
# from transformers.trainer_callback import TrainerCallback
# import os
# class PeftSavingCallback(TrainerCallback):
#     def on_train_end(self, args, state, control, **kwargs):
#         peft_model_path = os.path.join(state.best_model_checkpoint, "adapter_model")
#         kwargs["model"].save_pretrained(peft_model_path)

#         pytorch_model_path = os.path.join(state.best_model_checkpoint, "pytorch_model.bin")
#         os.remove(pytorch_model_path) if os.path.exists(pytorch_model_path) else None

In [ ]:
num_epochs = 3
batch_size = 32
training_args = TrainingArguments(
    output_dir=model_checkpoint+'1.3b',
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
    
)

In [27]:
torch.cuda.is_available()

True

In [28]:
print(torch.__version__)

2.0.0+cu117


In [31]:
is_torch_bf16_gpu_available()

NameError: name 'is_torch_bf16_gpu_available' is not defined

In [126]:
#data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [127]:
# setattr(model, 'model_parallel', True)
# setattr(model, 'is_parallelizable', True)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    tokenizer=tokenizer,
    
    #data_collator=data_collator,
    #callbacks=[PeftSavingCallback]
    #compute_metrics=compute_metrics,
)



In [129]:
# from accelerate import notebook_launcher
# def train_loop(batch_size=128, lr=0.001, num_epochs=3):
#     model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
#     peft_config = LoraConfig(
#         task_type="CAUSAL_LM", inference_mode=False, r=16, lora_alpha=16, lora_dropout=0.1, bias="all"
#     )
#     model = get_peft_model(model, peft_config)
#     model.print_trainable_parameters()
#     training_args = TrainingArguments(
#         output_dir=model_checkpoint+'peft',
#         learning_rate=lr,
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size,
#         num_train_epochs=num_epochs,
#         weight_decay=0.01,
#         evaluation_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=True,
    
#     )
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=lm_datasets["train"],
#         eval_dataset=lm_datasets["validation"],
#         tokenizer=tokenizer,
#         #data_collator=data_collator,
#         callbacks=[PeftSavingCallback]
#         #compute_metrics=compute_metrics,
#     )
#     trainer.train()
# notebook_launcher(train_loop, args={}, num_processes=2)

In [24]:
# model.to('cuda')

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.save_model("./my_model_1.3b")

In [ ]:
# model.save_pretrained("./output_dir_peft") 

In [138]:
# torch.cuda.empty_cache()

In [139]:
# torch.cuda.memory_allocated()

32702575616

In [72]:
# conf = '{"base_model_name_or_path":"facebook/galactica-125M", "task_type": "CAUSAL_LM", "inference_mode" : false, "r" : 16, "target_modules"  : ["q_proj", "v_proj"], "lora_alpha" : 16, "lora_dropout" : 0.1, "fan_in_fan_out": false, "bias":"all", "modules_to_save": null, "init_lora_weights":true}'

<PeftType.LORA: 'LORA'>

In [47]:
# # from transformers import AutoModelForSeq2SeqLM
# # from peft import PeftModel, PeftConfig

# #peft_model_id = "smangrul/twitter_complaints_bigscience_T0_3B_LORA_SEQ_2_SEQ_LM"
# peft_model_id = "/home/sslashinin/kovakimyan/facebook/galactica-125Mpeft/checkpoint-42735/model/"
# #config = PeftConfig.from_pretrained(peft_model_id)
# #config = PeftConfig.from_pretrained(conf)
# # model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
# # model = PeftModel.from_pretrained(model, peft_model_id)
# # tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
# #print(model.generate("There's a theorem in number theory with a proof that famously was too large to fit in the margin. The paper that proved that theore", top_p=0.7, max_length=200))
# model = PeftModel.from_pretrained(model, "/home/sslashinin/kovakimyan/facebook/galactica-125Mpeft/checkpoint-42735/model/" )
# model = model.to('cuda')
# model.eval()
# inputs = tokenizer("Suggestion for a paraconsistent logic book [START_REF]", return_tensors="pt")

# with torch.no_grad():
#   outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=50)
#   print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])
# # 'complaint'


Suggestion for a paraconsistent logic book  A l l. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 


In [59]:
# model = AutoModelForCausalLM.from_pretrained("/home/sslashinin/kovakimyan/my_model2")
# model = model.to('cuda')
# #inputs = tokenizer("Suggestion for a paraconsistent logic book", return_tensors="pt")
# input_text = "quick limits question [START_REF]"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

# outputs = model.generate(input_ids, max_new_tokens=300)
# print(tokenizer.decode(outputs[0]))

# input_text = "Can this series be expressed in closed form, and if so, what is it?"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

# outputs = model.generate(input_ids, max_new_tokens=300)
# print(tokenizer.decode(outputs[0]))

quick limits question [START_REF]   i n   t h e   p a p e r   A   C o u r s e   i n   A r i t h m e t i c   o f   t h e   R e a l   n u m b e r s   a n d   P r o b a b i l i t y,   V o l.   1,   S p r i n g e r - V e r l a g,   N e w   Y o r k,   1 9 8 8. 
 
 
 
 
 
 
 T h e   p r o b l e m   i s   t h a t   t h e   p r o b l e m   i s   n o t   s o l v e d   b y   t h e   a n s w e r,   b u t   i t   i s   s o l v e d   b y   t h e   a n s w e r.   I t   i s   a   s t a n d a r d   p r o b l e m   i n   t h a t   t h e   a n s w e r   i s   n o t   s o l v e d   b y   t h e   a n
Can this series be expressed in closed form, and if so, what is it? 
 
 
 T h e   a n s w e r   i s   n o.   T h e   s e t   o f   a l l   s e t s   o f   t h e   f o r m   $ \ { ( x, y ) \ i n \ m a t h b b { R } ^ 2 \ m i d   x & l t ; y \ } $   i s   a   s u b s e t   o f   $ \ m a t h b b { R } ^ 2 $.   T h e   s e t   o f   a l l   s e t s   o f   t h e   f o r m   $ \ { ( x, y ) \ i n \ m a t h b b { R 

In [69]:
# from peft import PeftModel, PeftConfig

# model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
# model = PeftModel.from_pretrained(model, peft_model_id)

ValueError: Can't find 'adapter_config.json' at '/home/sslashinin/kovakimyan/my_model_peft/tokenizer_config'

In [11]:
!python3 run.py

imported success!

Using the latest cached version of the module from /home/sslashinin/.cache/huggingface/modules/datasets_modules/datasets/flax-sentence-embeddings--stackexchange_math_jsonl/6d8702724ae9d14595d55b555648bb7a4909ce0441581f0c5a81e742633441a6 (last modified on Fri May  5 02:02:00 2023) since it couldn't be found locally at flax-sentence-embeddings/stackexchange_math_jsonl., or remotely on the Hugging Face Hub.
No config specified, defaulting to: stackexchange_math_jsonl/title_answer
Found cached dataset stackexchange_math_jsonl (/home/sslashinin/.cache/huggingface/datasets/flax-sentence-embeddings___stackexchange_math_jsonl/title_answer/1.1.0/6d8702724ae9d14595d55b555648bb7a4909ce0441581f0c5a81e742633441a6)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 25.61it/s]
dataset prepared success!

Map (num_proc=4):   0%|                       | 0/110095 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model h